In [1]:
import os
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
import seaborn as sns
import pingouin as pg

from matplotlib import cm
from matplotlib.colors import Normalize

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import statsmodels.api as sm

In [4]:
import pyreadr

In [5]:
from sklearn.linear_model import LinearRegression

## Load dataset:

In [6]:
F_ID = 9

if F_ID == 8:
    char_list_path = os.path.join(top_dir, "Data", "NTNU", "char_list.txt")
    
    with open(char_list_path, "r", encoding="utf-8") as f:
         char_list = [ char.replace('\n', '') for char in f.readlines() ]
            
    fn, dfn = {
        0: ("Data_all.RData", "DF"), 
        1: ("Data_all.z.RData", "DF.z"), 
        2: ("Data_clean.RData", "DF.clean"), 
        3: ("Data_clean.z.RData", "DF.clean.z"), 
        4: ("Data_else.RData", "DF.else")
    }[1]
            
else:
    fn = ""

top_dir = os.path.join("C:\\", "Users", "PinWei", "my_Haskins_project")

data_path = {
#     1: os.path.join(top_dir, "Data", "Tse_et_al", "Tse_et_al.xlsx"), 
#     2: os.path.join(top_dir, "Data", "Tse_et_al", "Chan and Tse.xlsx"), 
#     3: os.path.join(top_dir, "Data", "HK_norm_2021.xlsx"), 
#     4: os.path.join(top_dir, "Data", "LD2T2020_20240904.xlsx"), 
#     5: os.path.join(top_dir, "Data", "Tse_et_al", "CLP_data.xlsx"), 
    6: os.path.join(top_dir, "Data", "Tse_et_al", [
        "Tse_Yap_Chan_nz.csv", "Tse_Yap_Chan.csv"][1]), 
    7: os.path.join(top_dir, "Data", "Chang_et_al", [
        "Chang_Lee_2020.xlsx", "Chang_Lee_2020_z.xlsx"][1]), 
    8: os.path.join(top_dir, "Data", "NTNU", fn), 
    9: os.path.join(top_dir, "Stats", "NTNU", "Indv_slopes.csv")
}[F_ID]

if F_ID == 4:
    data = pd.read_excel(data_path, sheet_name=1)
elif F_ID == 6 or F_ID == 9:
    data = pd.read_csv(data_path)
elif F_ID == 8:
    data = pyreadr.read_r(data_path)[dfn]
else:
    data = pd.read_excel(data_path)

print(data_path)

C:\Users\PinWei\my_Haskins_project\Stats\NTNU\Indv_slopes.csv


In [7]:
print(data.columns)

Index(['SID', 'Sex', 'Grade', 'Score', 'logF', 'OPC', 'OSC', 'Imag'], dtype='object')


## Data pre-processing:

In [8]:
if F_ID == 5:
    cols = ["NS_1", "NS_2", "Phon_1", "Phon_2", 
            "Freq_C1", "Freq_C2", "Freq_word", 
            "REG.1_C1", "REG.1_C2", "REG.2_C1", "REG.2_C2", 
            "P.cons_1", "P.cons_2", "OP.cons_1", "OP.cons_2", 
            "HD_1", "HD_2", "NbS_1", "NbS_2", 
            "NMe_1", "NMe_2", "Trans_1", "Trans_2", 
            "Vale", "Arou", "Conc", "Fami", "Imag"]
    
elif F_ID == 6:
    cols = ['C1_Reg.1', 'C1_Reg.2', 'C2_Reg.1', 'C2_Reg.2', 
            'C1_St.num.Z', 'C2_St.num.Z', 
            'C1_logF.Z', 'C2_logF.Z', 'Word_logF.Z', # 'C_logF.sum.Z', 
            'C1_Ph.num.Z', 'C2_Ph.num.Z', 
            'C1_Homo.D.Z', 'C2_Homo.D.Z',
            'C1_Ph.cons.Z', 'C2_Ph.cons.Z', 
            'C1_OP.cons.Z', 'C2_OP.cons.Z', 
            'C1_A_Nb.num.Z', 'C2_B_Nb.num.Z', 
            'C1_A_Nb.logF.Z', 'C2_B_Nb.logF.Z', 
            'C1_S.num.Z', 'C2_S.num.Z', 
            'C1_S.Trans.Z', 'C2_S.Trans.Z', # 'C_S.T.avg.Z', 
            'C1_S.T.Nb.Z', 'C2_S.T.Nb.Z', 
            'Imag.Z']

elif F_ID == 7:
    data = data.rename(columns={"character": "Char"})
    cols = list(data.columns[14: ])
    
elif F_ID == 8:
    data = data.rename(columns={
        "OS.Con.type": "OSC", 
        "OP.Con.type": "OPC", 
    })

    cols = ["NS", "Freq", "logF", "HD", "P.Reg", "P.Unp", "OPC", 
            "OSC", "SAR", "Imag", "Conc", "Fami", "NM"]

#     for sc_add in ["w2v", "GloVe", "DSG"]:
#         cols.insert(cols.index("OSC")+1, "OSC." + sc_add)

    for col in cols:
        data[col] = data[col].astype(float)
        
elif F_ID == 9:
    cols = list(data.columns[3: ])
    
print(cols)

['Score', 'logF', 'OPC', 'OSC', 'Imag']


In [9]:
print(data.dtypes)

SID        int64
Sex       object
Grade      int64
Score      int64
logF     float64
OPC      float64
OSC      float64
Imag     float64
dtype: object


## Save description to file:

In [ ]:
data_desc = (data[cols].describe()
             .round(3)
             .loc[['count', 'max', 'min', 'mean', 'std'], :]
             .T)

In [ ]:
print(data_desc)

In [ ]:
if F_ID == 8: 
    desc_fn = (os.path.basename(data_path)
               .replace("Data_", "Desc_")
               .replace(".RData", ".xlsx"))
else:
    desc_fn = os.path.basename(data_path).split(".")[0]

desc_path = {
    6: os.path.join(top_dir, "Stats", "Tse_et_al", f"Desc_{desc_fn}.xlsx"), 
    7: os.path.join(top_dir, "Stats", "Chang_et_al", f"Desc_{desc_fn}.xlsx"), 
    8: os.path.join(top_dir, "Stats", "NTNU", desc_fn), 
    9: os.path.join(top_dir, "Stats", "NTNU", f"Desc_{desc_fn}.xlsx")
}[F_ID]

data_desc.to_excel(desc_path)
print(f"saved: {desc_path}")

## Plot data distribution:

In [10]:
if F_ID == 6: 
    except_cols = ['C1_Reg.1', 'C1_Reg.2', 'C2_Reg.1', 'C2_Reg.2']
    fd = os.path.join(top_dir, "Figs", "Tse_et_al")

elif F_ID == 7: 
    except_cols = ["REG", "UNP"]
    fd = os.path.join(top_dir, "Figs", "Chang_et_al")
    
elif F_ID == 8: 
    except_cols = ["P.Reg", "P.Unp"]
    fd = os.path.join(top_dir, "Figs", "NTNU", dfn)

In [ ]:
if (F_ID == 6) or if (F_ID == 7) or (F_ID == 8):  
    
    no_dup_char = data.drop_duplicates(subset = ["Char"])
    
    targ_cols = [ c for c in cols if c not in except_cols ]
    
    for targ_col in targ_cols:

        fn = f"[histplot] {targ_col}.png"
        if not os.path.exists(fd):
            os.makedirs(fd) 

        sns.histplot(
            data=no_dup_char, 
            x=targ_col, 
            binrange=(-3, 3), 
            kde=True, 
            bins=30
        )
        plt.tight_layout() 
        plt.savefig(os.path.join(fd, fn), format='png', dpi=200)
        plt.close()
        
        print(f"saved: {fn}")

In [ ]:
if F_ID == 8:  
    
    no_dup_char = data.drop_duplicates(subset = ["Char"])
    
    setA = no_dup_char.query("Char in @char_list")

    for targ_col in targ_cols:

        fn = f"[hist_compare] {targ_col}.png"
        if not os.path.exists(fd):
            os.makedirs(fd)
            
        setB = (no_dup_char
                .query("Char not in @char_list")
                .dropna(subset=[targ_col]))

        plt.style.use('seaborn-v0_8-white')
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.hist([setA[targ_col], setB[targ_col]], 
                bins=30, 
                range=[-3, 3], 
                color=['b', 'orange'], 
                alpha=0.8, 
                label=[f"{len(setA)} char", f"{len(setB)} char"])
        ax.set_xlabel(targ_col, fontsize=16)
        ax.set_ylabel("Count", fontsize=16)
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        plt.legend(loc='upper right', fontsize=14)
        plt.tight_layout() 
        plt.savefig(os.path.join(fd, fn), format='png', dpi=200)
        plt.close()
        
        print(f"saved: {fn}")

## Plot correlation matrix and save:

In [10]:
if F_ID == 8: 
    desc_fn = (os.path.basename(data_path)
               .replace("Data_", "[cormat] ")
               .replace(".RData", ".png"))
else:
    desc_fn = os.path.basename(data_path).split(".")[0]

cormat_path = {
    6: os.path.join(top_dir, "Figs", "Tse_et_al", f"[cormat] {desc_fn}.png"), 
    7: os.path.join(top_dir, "Figs", "Chang_et_al", f"[cormat] {desc_fn}.png"), 
    8: os.path.join(top_dir, "Figs", "NTNU", desc_fn), 
    9: os.path.join(top_dir, "Figs", "NTNU", f"Slope cormat.png")
}[F_ID]

print(cormat_path)

C:\Users\PinWei\my_Haskins_project\Figs\Chang_et_al\[cormat] Chang_Lee_2020_z.png


In [11]:
PLOT_SIG = 0

# ---

df_select = data.loc[:, cols]
r_raw = df_select.corr()
p_raw = df_select.corr(method=lambda x, y: stats.pearsonr(x, y)[1]) - np.eye(*r_raw.shape)

if PLOT_SIG:
    n_of_comp = np.sum([ n for n in range(len(cols)) ])
    corrected_thres = [ alpha/n_of_comp for alpha in [.05, .01, .001] ]
    p_sig = p_raw.applymap(lambda x: "".join(['*' for thres in corrected_thres if x <= thres]))
    r_sig = r_raw.round(2).astype(str) + p_sig
else:
    r_sig = r_raw.round(2).astype(str)

# print(round(p_raw, 5))

In [12]:
mask = np.zeros_like(r_raw)
mask[np.triu_indices_from(mask)] = True

sns.set(style="white")

if F_ID == 7: 
    fig, ax = plt.subplots(figsize=(8, 6), dpi=200)
    fz, xr = 16, 0
    
elif F_ID == 8: 
    fig, ax = plt.subplots(figsize=(12, 8), dpi=200)
    fz, xr = 16, 0
    
elif F_ID == 9: 
    fig, ax = plt.subplots(figsize=(5, 3), dpi=200)
    fz, xr = 12, 0

sns.heatmap(
    data=r_raw, mask=mask, 
    square=False, vmin=-1, vmax=1, 
    cmap="RdBu_r", cbar=False, 
    annot=r_sig, fmt="", annot_kws={'size': fz}, 
    linewidth=.5
)
ax.set_xticklabels(cols, rotation=xr, fontsize=fz)
ax.set_yticklabels(cols, rotation=0, fontsize=fz)
plt.tight_layout() 
plt.savefig(cormat_path, format='png')
plt.close()

print(f"saved: {cormat_path}")

saved: C:\Users\PinWei\my_Haskins_project\Figs\Chang_et_al\[cormat] Chang_Lee_2020_z.png


In [ ]:
if F_ID == 9: 
    
    out_path = os.path.join(
        top_dir, "Stats", "NTNU", "Indv_slopes_corr.csv")

    cols = data.columns[3:]
    
    corrs = {}
    for gd in range(1, 7):
        df_select = data.query("Grade == @gd").loc[:, cols]
        corr_matrix = df_select.corr().to_numpy()
        corr_vector = corr_matrix[np.triu_indices_from(corr_matrix, k=1)]
        corrs[f"G{gd}"] = corr_vector

    corr_names = []
    for x, c1 in enumerate(cols):
        for c2 in cols[x+1::]:
            corr_names.append(f"{c1} × {c2}")

    slopes_corr = pd.DataFrame(corrs, index = corr_names)
    slopes_corr.to_csv(out_path)
    
    print(f"saved: {out_path}")

In [ ]:
if F_ID == 9: 
    
    for gd in range(1, 7):
        out_path = os.path.join(top_dir, "Figs", "NTNU", 
                                f"slope cormat (G-{gd}).png") 

        df_select = data.query("Grade == @gd").loc[:, cols]

        r_raw = df_select.corr()
        p_raw = df_select.corr(method=lambda x, y: stats.pearsonr(x, y)[1]) - np.eye(*r_raw.shape)
        p_sig = p_raw.applymap(lambda x: "".join(['*' for thres in corrected_thres if x <= thres]))
        r_sig = r_raw.round(2).astype(str) + p_sig

        mask = np.zeros_like(r_raw)
        mask[np.triu_indices_from(mask)] = True

        fig, ax = plt.subplots(figsize=(5, 3), dpi=200)
        fz = 12 
        sns.heatmap(
            data = r_raw, mask=mask, 
            square=False, vmin=-1, vmax=1, 
            cmap="RdBu_r", cbar=False, 
            annot=r_sig, fmt="", linewidth=.5
        )
        ax.set_xticklabels(cols, rotation=0, fontsize=fz)
        ax.set_yticklabels(cols, rotation=0, fontsize=fz)
        
        plt.tight_layout() 
        plt.savefig(out_path, format='png')
        plt.close()
        
        print(f"saved: {out_path}")

## Line plot (show change across grades):

In [ ]:
if F_ID == 9: 
    
    out_path = os.path.join(
        top_dir, "Figs", "NTNU", "Slope corrs change across grades.png")
    
    fig = plt.figure(figsize=(8, 8))

    ax1 = fig.add_subplot(211)
    df = slopes_corr.iloc[:4, :]
    for idx, row in df.iterrows():
        ax1.plot(df.columns, row, 
                 marker='o', label=idx)
    ax1.grid(True)
    ax1.set_ylabel('Correlation coefficient', size=16)
    plt.legend(loc='lower right')

    ax2 = fig.add_subplot(212)
    df = slopes_corr.loc[["OPC × Imag", "OPC × OSC"], :]
    for idx, row in df.iterrows():
        ax2.plot(df.columns, row, 
                 marker='o', label=idx)
    ax2.grid(True)
    ax2.set_xlabel('Grades', size=16)
    ax2.set_ylabel('Correlation coefficient', size=16)
    plt.legend(loc='best')
    
    plt.tight_layout() 
    plt.savefig(out_path, format='png')
    plt.close()
    
    print(f"saved: {out_path}")

## Scatter plot:

In [ ]:
# var_x = ["Imag", "OSC"][0]
# var_y = "OPC"

# r, p = stats.pearsonr(data[var_x], data[var_y])
# print(f"Correlation between {var_x} and {var_y} is {r:.2f} (p = {p:.3f})")

In [ ]:
if F_ID == 9: 
    
    c_min = data["Score"].min()
    c_max = data["Score"].max()
    
    for var_x, x_name in [("Imag", "Imageability"), ("OSC", "OSC")]:
        var_y, y_name = ("OPC", "OPC")

        out_path = os.path.join(top_dir, "Figs", "NTNU", 
                                f"[scatter] {var_x} × {var_y}.png")

        data["Grade"] = data["Grade"].astype('category')
        markers = ['o', 's', 'D', '^', 'v', 'P']

        X = data[var_x].values.reshape(-1, 1)
        y = data[var_y].values
        reg = LinearRegression().fit(X, y)

        y_pred = reg.predict(X)
        slope = reg.coef_[0]
        intercept = reg.intercept_
        equation = f"y = {slope:.2f}x + {intercept:.2f}"

        sns.set(style="whitegrid")
        fig, ax = plt.subplots(figsize=(10, 8))

        sns.scatterplot(
            data=data, 
            ax=ax, 
            x=var_x, y=var_y, 
            hue="Score", style="Grade", 
            hue_norm=(c_min, c_max), 
            markers=markers, 
            palette='viridis', 
            legend=False, 
            alpha=.7, 
            s=100
        )
        ax.plot(
            data[var_x], y_pred, 
            color='black', linestyle='--', linewidth=1
        )
        ax.text(
            0.5, 0.9, equation, 
            horizontalalignment='center', 
            verticalalignment='center', 
            transform=plt.gca().transAxes, 
            fontsize=16, color='black'
        )

        norm = Normalize(vmin=c_min, vmax=c_max)
        sm = plt.cm.ScalarMappable(cmap='viridis', norm=norm)
        sm.set_array([])
        cbar = fig.colorbar(sm, ax=ax)
        cbar.set_label("Naming score")

        unique_grades = data["Grade"].unique()
        grade_handles = [ plt.Line2D([0], [0], marker=markers[i], 
                                     color='w', markerfacecolor='gray', markersize=10) 
                          for i, grade in enumerate(unique_grades) ]
        ax.legend(grade_handles, unique_grades, title='Grade', fontsize=14)

        ax.set_xlabel(f"Sensivity to {x_name}", fontsize=16)
        ax.set_ylabel(f"Sensivity to {y_name}", fontsize=16)
        ax.tick_params(axis='both', which='major', labelsize=16)

        plt.tight_layout() 
        plt.savefig(out_path, dpi=300, bbox_inches='tight') 
        plt.close()
        
        print(f"saved: {out_path}")

In [13]:
# 

In [15]:
scale = ["fixed", "flexible", "ranked"][-1]

if F_ID == 9: 
    
    hue_from = "Score"
    
    if scale == "fixed":
        c_min = data["Score"].min() 
        c_max = data["Score"].max()
        
    for var_x, x_name in [("Imag", "Imageability"), ("OSC", "OSC")]:
        var_y, y_name = ("OPC", "OPC")

        for gd in range(1, 7):

            df_select = data.query("Grade == @gd").loc[:, cols]
            
            if scale == "ranked":
                df_select["Score_Rank"] = df_select["Score"].rank(axis=0)
                c_min = df_select["Score_Rank"].min()
                c_max = df_select["Score_Rank"].max()
                hue_from = "Score_Rank"
                
            elif scale == "flexible":
                c_min = df_select["Score"].min()
                c_max = df_select["Score"].max()

            fig_name = f"[{c_min} ~ {c_max}] {var_x} × {var_y} (G-{gd}).png"
            # fig_name = f"[scatter] {var_x} × {var_y} (G-{gd}).png"
            
            out_path = os.path.join(top_dir, "Figs", "NTNU", fig_name)

            X = df_select[var_x].values.reshape(-1, 1)
            y = df_select[var_y].values
            reg = LinearRegression().fit(X, y)

            y_pred = reg.predict(X)
            slope = reg.coef_[0]
            intercept = reg.intercept_
            equation = f"y = {slope:.2f}x + {intercept:.2f}"

            sns.set(style="whitegrid")
            fig, ax = plt.subplots(figsize=(8, 6))

            sns.scatterplot(
                data=df_select, 
                ax=ax, 
                x=var_x, y=var_y, 
                hue=hue_from, 
                hue_norm=(c_min, c_max), 
                marker='o', 
                palette='viridis', 
                legend=False, 
                alpha=.7, 
                s=100
            )
            ax.plot(
                df_select[var_x], y_pred, 
                color='black', linestyle='-', linewidth=1
            )
            ax.text(
                0.5, 0.9, equation, 
                horizontalalignment='center', 
                verticalalignment='center', 
                transform=plt.gca().transAxes, 
                fontsize=16, 
                color='black'
            )
            ax.axvline(
                x=0, color="darkgray", linewidth=2, linestyle="--"
            )
            ax.axhline(
                y=0, color="darkgray", linewidth=2, linestyle="--"
            )
            
            norm = Normalize(vmin=c_min, vmax=c_max)
            sm = plt.cm.ScalarMappable(cmap='viridis', norm=norm)
            sm.set_array([])
            cbar = fig.colorbar(sm, ax=ax)
            cbar.set_label("Naming score")

            ax.set_xlabel(f"Sensivity to {x_name}", fontsize=16)
            ax.set_ylabel(f"Sensivity to {y_name}", fontsize=16)
            ax.tick_params(axis='both', which='major', labelsize=16)

            plt.tight_layout() 
            plt.savefig(out_path, dpi=300, bbox_inches='tight') 
            plt.close()
            
            print(f"saved: {out_path}")

saved: C:\Users\PinWei\my_Haskins_project\Figs\NTNU\[1.0 ~ 101.0] Imag × OPC (G-1).png
saved: C:\Users\PinWei\my_Haskins_project\Figs\NTNU\[1.0 ~ 116.0] Imag × OPC (G-2).png
saved: C:\Users\PinWei\my_Haskins_project\Figs\NTNU\[1.0 ~ 123.0] Imag × OPC (G-3).png
saved: C:\Users\PinWei\my_Haskins_project\Figs\NTNU\[1.0 ~ 134.0] Imag × OPC (G-4).png
saved: C:\Users\PinWei\my_Haskins_project\Figs\NTNU\[1.0 ~ 151.0] Imag × OPC (G-5).png
saved: C:\Users\PinWei\my_Haskins_project\Figs\NTNU\[1.0 ~ 104.0] Imag × OPC (G-6).png
saved: C:\Users\PinWei\my_Haskins_project\Figs\NTNU\[1.0 ~ 101.0] OSC × OPC (G-1).png
saved: C:\Users\PinWei\my_Haskins_project\Figs\NTNU\[1.0 ~ 116.0] OSC × OPC (G-2).png
saved: C:\Users\PinWei\my_Haskins_project\Figs\NTNU\[1.0 ~ 123.0] OSC × OPC (G-3).png
saved: C:\Users\PinWei\my_Haskins_project\Figs\NTNU\[1.0 ~ 134.0] OSC × OPC (G-4).png
saved: C:\Users\PinWei\my_Haskins_project\Figs\NTNU\[1.0 ~ 151.0] OSC × OPC (G-5).png
saved: C:\Users\PinWei\my_Haskins_project\Figs\N

## Modeling:

In [ ]:
# data_y = data[["Acc", "zRT"][1]]
# data_x = {
#     0: data.iloc[:, 9:],
#     1: data.iloc[:, 9:22],                          # phoneme features
#     2: data.iloc[:, [22, 23, 26, 27, 28]],          # orthographic features
#     3: data.iloc[:, [24, 25]+list(range(29, 39))],  # phonological features
#     4: data.iloc[:, -6:]                            # semantic features
# }[2]

In [ ]:
# data_x = sm.add_constant(data_x)
# results = sm.OLS(data_y, data_x).fit()
# # results = sm.GLM(data_y, data_x).fit()
# results.summary()